# SI 330: Data Manipulation 
## 17 - SQL

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

# <FONT COLOR="red">NOTE: Please install the ```pymysql``` module (conda install pymysql)

# SQL Outline

## Day 1

* AWS Overview
  * EC2, RDS

* Tasks
  * log into AWS Console
  * create EC2 instance
    * yum install mysql
    * wget .sql
    
## Day 2

* RDS
* MySQL

* Tasks
  * create RDS instance
  * load SQL file from EC2
  * browse with MySQLWorkbench
  
## Day 3

* pymysql

* Tasks
  * query goodreads
  
## Day 4

* BigQuery, Homework


## SQL

In general, we are going to:
```
SELECT some_fields FROM some_table
WHERE some_condition_exists
[ GROUP BY some_field ]
[ ORDER BY some_field ]
```

For example:
```
SELECT authors FROM books;
```
**Note:** the convention is to use UPPERCASE for SQL reserved words, and lowercase for the specifics of your instance

If we want all the fields:
```
SELECT * FROM books;
```

Adding a condition:
```
SELECT * FROM books
WHERE original_publication_year >= 2008;
```

And you can also group by a field:
```
SELECT original_publication_year,COUNT(*) FROM books
GROUP BY original_publication_year;
```

## Introduction to pymysql



```pymysql``` is a python interface to SQL servers (including MySQL, sqlite, etc.).  There are other interfaces (such as SQLAlchemy), but pymysql is the most straight-forward to work with. (Note caveats that we'll discuss in class.)

In [1]:
import pymysql

Fill in the ```host```, ```user``` and ```password``` fields in the next code block.  The host is the RDS endpoint; user should probably be your uniqname, and password should be whatever you set you database password to.

Run the block to establish a connection to your RDS-based MySQL server:

In [ ]:
connection = pymysql.connect(host='RDS endpoint', # remove this before submitting
                             user='username', # remove this before submitting
                             password='password', # remove this before submitting
                             db='goodreads',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

Here's a "pythonic" way to query your database.  THe following is a bit 

In [5]:
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM books WHERE original_title = 'The Hunger Games'")
    results = cursor.fetchall()
    for result in results:
        print(result['original_title'],result['authors'])

The Hunger Games Suzanne Collins


### COUNT

In [6]:
with connection.cursor() as cursor:
    cursor.execute("SELECT COUNT(*) FROM books WHERE authors = 'Suzanne Collins'")
    result = cursor.fetchone()
    print(result)

{'COUNT(*)': 9}


We can rename awkward column names by using the ```AS``` operator:

In [7]:
with connection.cursor() as cursor:
    cursor.execute("SELECT COUNT(*) AS cnt FROM books WHERE authors = 'Suzanne Collins'")
    result = cursor.fetchone()
    print(result)

{'cnt': 9}


### DISTINCT

Sometimes, we want to know the number of distinct (unique) values of a field.  Here are two ways of doing this:

In [8]:
with connection.cursor() as cursor:
    cursor.execute('SELECT DISTINCT authors FROM books')
    results = cursor.fetchall()
    print(len(results))

4669


In [12]:
with connection.cursor() as cursor:
    cursor.execute('SELECT COUNT(DISTINCT authors) FROM books')
    result = cursor.fetchone()
    print(result)

{'COUNT(DISTINCT authors)': 4669}


### <font color="magenta">Q1: Which of the above two ways of finding distinct values is more efficient?  Why?</font>

## SQL from pandas

We are going to continue to use our connection object from above in this section.  Pandas plays nicely with pymysql.

In [13]:
import pandas as pd

Just like what we found with read_csv (a relief compared to CSVDictReader), we have a read_sql method that allows us to create a dataframe based on an SQL query:

In [14]:
df = pd.read_sql('SELECT * FROM books',con=connection)
len(df)

10000

In [15]:
df.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780320e+12,Stephenie Meyer,2005,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780060e+12,Harper Lee,1960,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780740e+12,F. Scott Fitzgerald,1925,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


Let's work in pandas, and explore wildcards and regular expressions in SQL:

In [16]:
df = pd.read_sql('SELECT * FROM books WHERE original_title = "The Hunger Games"',con=connection)
df

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


In SQL, ```%``` means 1 or more wildcard characters; ```?``` means exactly one wildcard character.  So, if we wanted to look for titles that contain "Hunger Games", we could use:

In [17]:
df = pd.read_sql('SELECT * FROM books WHERE original_title LIKE "%Hunger Games%"',con=connection)
df

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,507,7938275,7938275,11349083,78,545265355,9.780550e+12,Suzanne Collins,2010,The Hunger Games Box Set,...,129665,159760,6429,1015,2521,13438,42932,99854,https://images.gr-assets.com/books/1360094673m...,https://images.gr-assets.com/books/1360094673s...


More powerfully, we could use regular expressions:

In [18]:
df = pd.read_sql('SELECT * FROM books WHERE original_title REGEXP ".*Hunger Games.*"',con=connection)
df

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,507,7938275,7938275,11349083,78,545265355,9.780550e+12,Suzanne Collins,2010,The Hunger Games Box Set,...,129665,159760,6429,1015,2521,13438,42932,99854,https://images.gr-assets.com/books/1360094673m...,https://images.gr-assets.com/books/1360094673s...


In [19]:
df = pd.read_sql('SELECT * FROM books WHERE original_publication_year IN (2008, 2010)',con=connection)
df.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,20,7260188,7260188,8812783,239,439023513,9.780440e+12,Suzanne Collins,2010,Mockingjay,...,1719760,1870748,96274,30144,110498,373060,618271,738775,https://images.gr-assets.com/books/1358275419m...,https://images.gr-assets.com/books/1358275419s...
2,56,1162543,1162543,2960529,183,031606792X,9.780320e+12,Stephenie Meyer,2008,Breaking Dawn,...,1070245,1107709,44550,100994,114893,213402,267706,410714,https://images.gr-assets.com/books/1361039438m...,https://images.gr-assets.com/books/1361039438s...
3,73,1656001,1656001,3328799,161,316068047,9.780320e+12,Stephenie Meyer,2008,The Host,...,749780,777560,39778,44215,62501,154906,227180,288758,https://images.gr-assets.com/books/1318009171m...,https://images.gr-assets.com/books/1318009171s...
4,88,6442769,6442769,3364505,178,014241493X,9.780140e+12,John Green,2008,Paper Towns,...,461311,655271,42717,14105,47183,154479,223895,215609,https://images.gr-assets.com/books/1349013610m...,https://images.gr-assets.com/books/1349013610s...


Sometimes, it's nicer to wrap our SQL across multiple lines, so we can use docstrings ("""...""") to create our SQL.

Here's an example of a join using a WHERE statement:

In [20]:
df = pd.read_sql("""
                SELECT books.original_title, ratings.rating 
                FROM books,ratings 
                WHERE books.authors LIKE '%Rowling%' 
                AND books.book_id = ratings.book_id;
                """, con = connection)   

In [21]:
df.shape

(136631, 2)

In [22]:
df.head()

,original_title,rating
0,Harry Potter and the Philosopher's Stone,5
1,Harry Potter and the Philosopher's Stone,4
2,Harry Potter and the Philosopher's Stone,4
3,Harry Potter and the Philosopher's Stone,5
4,Harry Potter and the Philosopher's Stone,3


In [23]:
df.sample(10)

,original_title,rating
100765,Harry Potter and the Half-Blood Prince,2
87218,Harry Potter and the Deathly Hallows,5
102358,Harry Potter and the Half-Blood Prince,5
117328,The Cuckoo's Calling,5
6327,Harry Potter and the Philosopher's Stone,5
60737,Harry Potter and the Chamber of Secrets,4
82486,Harry Potter and the Goblet of Fire,5
74603,Harry Potter and the Goblet of Fire,5
27812,Harry Potter and the Prisoner of Azkaban,4
133351,Career of Evil,3


We can leverage the groupby function in pandas to do helpful calculations:

In [24]:
df.groupby(by='original_title')[['rating']].mean()

,rating
original_title,
,4.588123
Career of Evil,4.134006
Complete Harry Potter Boxed Set,4.659530
Fantastic Beasts and Where to Find Them,3.775901
Fantastic Beasts and Where to Find Them: The Original Screenplay,4.384259
Harry Potter Boxed Set Books 1-4,4.605357
"Harry Potter Collection (Harry Potter, #1-6)",4.699571
Harry Potter and the Chamber of Secrets,4.229418
"Harry Potter and the Cursed Child, Parts One and Two",3.694737


Here's a schematic representation of the goodreads database:

![](assets/goodreadsEER.png)

In [25]:
df.sample(10)

,original_title,rating
41938,Harry Potter and the Order of the Phoenix,3
21577,Harry Potter and the Philosopher's Stone,5
125164,The Tales of Beedle the Bard,3
74945,Harry Potter and the Goblet of Fire,5
116305,The Cuckoo's Calling,4
59418,Harry Potter and the Chamber of Secrets,5
125413,The Tales of Beedle the Bard,3
86057,Harry Potter and the Deathly Hallows,5
40063,Harry Potter and the Order of the Phoenix,4
106934,Harry Potter and the Half-Blood Prince,5


### <font color="magenta">Q2: How many books in our dataset were originally published in 2016?</font>

In [29]:
# insert your code here

### <font color="magenta">Q3: How many books were published in each year of original publication in our dataset?</font>


In [29]:
# insert your code here

### <font color="magenta">Q4: Do you notice anything interesting or strange about your results from the previous question?  Investigate at least one of these anomalies using either an SQL query or pandas filtering. </font>

Insert your answer here

In [32]:
# insert your code here

### <font color="magenta">Q5: Create a dataframe that contains a list of all book titles from 2017 as well as the tag_names associated with each book.</font>

In [33]:
# Insert your code here

# END OF NOTEBOOK: REMEMBER TO STRIP IDETIFYING INFORMATION FROM TOP OF NOTEBOOK!
Please submit both ipynb and html formats of this notebook to Canvas.